In [4]:
pip install streamlit

     |████████████████████████████████| 7.2MB 2.6MB/s 
     |████████████████████████████████| 112kB 43.9MB/s 
     |████████████████████████████████| 102kB 9.5MB/s 
     |████████████████████████████████| 4.4MB 37.6MB/s 
     |████████████████████████████████| 122kB 25.1MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp36-none-any.whl size=13450 sha256=2bde1e1f036819005ecf1af2d9bf35c2fac342c6027605a7987bc7d449b72ffc
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=43715367ae6e503ccf054295a7c226d3164de08398c43b405e539856761bc565
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=d076a39e9a11070a3263a149f43ad6820c8e58f6f1b20cf9afd672f2afe4fbc6
  Stored in directory: /root/.cache/pip/wheel

In [20]:
+streamlit
+pyngrok
+nohub/background

In [20]:
!pip install streamlit

In [21]:
!pip install pyngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.10-cp36-none-any.whl size=16378 sha256=a92921f9ffbdb914db2b3ef529585dea59db7bc7e40269028cebd00199fb0720
  Stored in directory: /root/.cache/pip/wheels/0d/7f/06/e8079dd6f80eb6683fac47c4e1cb720b61cd82d6e4c13d87a5
Successfully built pyngrok


In [40]:
!wget https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt

--2020-09-08 14:03:42--  https://raw.githubusercontent.com/TanmayKhot/Fluorine/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘imagenet_classes.txt.1’

imagenet_classes.tx 100%[===================>]  21.17K  --.-KB/s    in 0.01s   

2020-09-08 14:03:42 (1.46 MB/s) - ‘imagenet_classes.txt.1’ saved [21675/21675]



In [47]:
%%writefile main.py
import streamlit as st
PAGE_CONFIG = {"page_title":"Image Classifier Visualizer with Adv Attacks.io", "page_icon":":smiley:", "layout":"centered"}
st.beta_set_page_config(**PAGE_CONFIG)
import time
import torch
import torchvision.models as models
from PIL import Image
import joblib

with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]
st.set_option('deprecation.showfileUploaderEncoding', False)
st.title("Adversarial Attacks Visualizer")
import base64

def get_base64_of_bin_file(bin_file):
    with open(bin_file, 'rb') as f:
        data = f.read()
    return base64.b64encode(data).decode()

def set_png_as_page_bg(jpg_file):
    bin_str = get_base64_of_bin_file(jpg_file)
    page_bg_img = '''
    <style>
    body {
    background-image: url("data:image/jpg;base64,%s");
    background-size: cover;
    }
    </style>
    ''' % bin_str
    st.markdown(page_bg_img, unsafe_allow_html=True)
    return

set_png_as_page_bg('deep-learning-neural-networks.jpg')

def classify(model, image, classes):
  model.eval()
  from torchvision import transforms
  
  preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])
  input_tensor = preprocess(image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

  # move the input and model to GPU for speed if available
  if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')
  with torch.no_grad():
    output = model(input_batch)
  _, index = torch.max(output, 1)
  percentage = torch.nn.functional.softmax(output, dim=1)[0] * 100
  st.write(classes[index[0]],percentage[index[0]].item())

select =  st.sidebar.selectbox('Please enter your choice :',('Home','Image Classifier','Image Classifier with Adversarial Attack Visualizer'))
if select == 'Image Classifier':
    model_select = st.sidebar.selectbox('Please select a Model from the list',('','AlexNet', 'DenseNet', 'ResNet','VGG'))
    st.header('Please enter the image for classification')
    opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))

    if opt == 'Upload image from device':
    
        file = st.file_uploader('Select', type = ['jpg', 'png'])

        st.set_option('deprecation.showfileUploaderEncoding', False)
        if file is not None:
            image = Image.open(file)
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
    if opt == 'Upload image via link':
        img = st.text_input('Enter Link for the image')
        if st.button('Submit'):
            import urllib.request
            image = Image.open(urllib.request.urlopen(img))
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            

    if opt == "Please Select":
        pass

if select == 'Image Classifier with Adversarial Attack Visualizer':    
    model_select = st.sidebar.selectbox('Please select a Model from the list:',('','AlexNet', 'DenseNet', 'ResNet','VGG'))
    attack_select = st.sidebar.selectbox('Please select an Adversarial Attack from the list:',('','Adv1','Adv2')) 
    num = st.sidebar.number_input('Enter Epsilon value:')
    st.subheader('Result after adding 1.0 is: ')
    st.subheader(num+1.0)
    st.header('Please enter the image for classification')
    opt = st.selectbox("How do you want to upload the image for classification?\n", ('Please Select', 'Upload image via link', 'Upload image from device'))

    if opt == 'Upload image from device':
    
        file = st.file_uploader('Select', type = ['jpg', 'png'])

        st.set_option('deprecation.showfileUploaderEncoding', False)
        if file is not None:
            image = Image.open(file)
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
    if opt == 'Upload image via link':
        img = st.text_input('Enter Link for the image')
        if st.button('Submit'):
            import urllib.request
            image = Image.open(urllib.request.urlopen(img))
            if model_select == 'VGG':
              model = models.vgg16(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'AlexNet':
              model = models.alexnet(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'DenseNet':
              model = models.densenet121(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
            elif model_select == 'ResNet':
              model = models.resnet101(pretrained=True)
              st.image(image, width = 300, caption = 'Uploaded Image')
              classify(model, image, classes)
    if opt == "Please Select":
        pass

if select == 'Home':
    st.balloons()
    html_temp = '''
    <div style = "background-color: rgba(25,25,112,0.20); padding: 20px; padding-bottom: 20px; padding-left: 20px; padding-right: 20px">
    <h2>Hey there! Welcome to Adversarial Attacks Visualizer.</h2>
    </div>
    '''
    st.markdown(html_temp, unsafe_allow_html=True)

Overwriting main.py


In [24]:
!ls

deep-learning-neural-networks.jpg  main.py  sample_data


In [25]:
!ngrok authtoken 1hE95HY9G1doEqBjcAX1YYkmGLs_4bUKc6kusoP8iGEQRKjfi

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [26]:
from pyngrok import ngrok

In [28]:
!streamlit run main.py&>/dev/null&

In [29]:
!pgrep streamlit
 

517
550
967


In [30]:
publ_url = ngrok.connect(port='8501')

2020-09-08 11:56:51.344 INFO    pyngrok.process: ngrok process starting: 1002
2020-09-08 11:56:51.362 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="no configuration paths supplied"

2020-09-08 11:56:51.364 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml

2020-09-08 11:56:51.367 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil

2020-09-08 11:56:51.373 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040

2020-09-08 11:56:51.504 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="tunnel session started" obj=tunnels.session

2020-09-08 11:56:51.505 INFO    pyngrok.process: t=2020-09-08T11:56:51+0000 lvl=info msg="client session established" obj=csess id=534c65819f79

2020-09-08 11:56:51.520 INFO    pyngrok.process: ngrok process has 

In [31]:
publ_url

'http://1b404d89d6e7.ngrok.io'